<a href="https://colab.research.google.com/github/raulc66/AI-Learning/blob/main/Transfer_Learning_for_Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 : Importing the Libraries

In [1]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Step 2 : Importing the dataset from Kaggle

In [4]:
# install Kaggle API
! pip install -q  kaggle

In [5]:
# create a directory as kaggle
!mkdir -p ~/.kaggle

In [6]:
# importing the Kaggle API key to google colab
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [7]:
# copy the API key to kaggle directory
! cp kaggle.json ~/.kaggle

In [8]:
# disable the API key
! chmod 600 /root/.kaggle/kaggle.json

In [9]:
# import the dataset
! kaggle datasets download -d tongpython/cat-and-dog

Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
 91% 198M/218M [00:01<00:00, 222MB/s]
100% 218M/218M [00:01<00:00, 204MB/s]


In [10]:
# unzipping the dataset
! unzip -q /content/cat-and-dog.zip

In [33]:
training_dir = '/content/training_set/training_set'
test_dir = '/content/test_set/test_set'

# Step 3 : Building the Model

In [34]:
# reshape the images
img_shape = (128,128,3) # the pretrained model expects this format

## Loading the Pre-Trained Model(MobileNetV2)

In [35]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, include_top=False, weights='imagenet') # include_top = the first of the model is not included

In [36]:
base_model.summary()

Model: "mobilenetv2_1.00_128"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 128, 128, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 64, 64, 32)     │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 64, 64, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 64, 64, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 64, 64, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 64, 64, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 64, 64, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 64, 64, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 64, 64, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 64, 64, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 65, 65, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 32, 32, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [37]:
# freezing the model
base_model.trainable = False # When not frozen, upon training, the weights of the model change | Only custom layers will be trained , not the entire network

## Defining the custom head for the network

In [38]:
base_model.output

<KerasTensor shape=(None, 4, 4, 1280), dtype=float32, sparse=False, name=keras_tensor_309>

In [39]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [40]:
global_average_layer

<KerasTensor shape=(None, 1280), dtype=float32, sparse=False, name=keras_tensor_310>

In [41]:
# output layer / prediction layer
prediction_layer = tf.keras.layers.Dense(units=1, activation= 'sigmoid')(global_average_layer)

## Define the transfer learning model

In [42]:
# define an object
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [43]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 128, 128, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 64, 64, 32)     │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 64, 64, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 64, 64, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 64, 64, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 64, 64, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 64, 64, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 64, 64, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 64, 64, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 64, 64, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 65, 65, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 32, 32, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [44]:
# compiling the model
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [45]:
model.compile(optimizer=opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

## Create Data Generators (Data Preprocessing)

In [46]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_gen_train = ImageDataGenerator(rescale = 1/255.0)
data_gen_test = ImageDataGenerator(rescale = 1/255.0)

In [47]:
train_generator = data_gen_train.flow_from_directory(directory = training_dir, target_size=(128,128), batch_size = 128, class_mode = 'binary')

Found 8005 images belonging to 2 classes.


In [48]:
test_generator = data_gen_test.flow_from_directory(directory = test_dir, target_size=(128,128), batch_size = 128, class_mode = 'binary')

Found 2023 images belonging to 2 classes.


# Step 4 : Training the model

In [51]:
model.fit(train_generator, epochs = 5, validation_data = test_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - accuracy: 0.6561 - loss: 0.6297 - val_accuracy: 0.8433 - val_loss: 0.4041
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.8566 - loss: 0.3693 - val_accuracy: 0.9041 - val_loss: 0.2684
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 210s 3s/step - accuracy: 0.9111 - loss: 0.2583 - val_accuracy: 0.9293 - val_loss: 0.2004
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 160s 3s/step - accuracy: 0.9286 - loss: 0.1981 - val_accuracy: 0.9441 - val_loss: 0.1627
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9422 - loss: 0.1627 - val_accuracy: 0.9501 - val_loss: 0.1405


# Step 5 : Fine Tuning

In [66]:
# unfreeze the model
base_model.trainable = True

In [67]:
len(base_model.layers)

154

In [68]:
fine_tune_at = 100

In [69]:
# freeze the layers before 100
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [70]:
opt_2 = tf.keras.optimizers.RMSprop(learning_rate=0.0001) # to avoid errors, use another instance for the optimizer

In [71]:
# compile the model
model.compile(optimizer=opt_2, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [74]:
#tf.config.run_functions_eagerly(False)

In [75]:
# train the model
model.fit(train_generator, epochs = 5, validation_data=(test_generator))

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 240s 4s/step - accuracy: 0.9933 - loss: 0.0191 - val_accuracy: 0.9560 - val_loss: 0.2035
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 242s 4s/step - accuracy: 0.9999 - loss: 0.0045 - val_accuracy: 0.9723 - val_loss: 0.1101
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 228s 4s/step - accuracy: 0.9998 - loss: 0.0018 - val_accuracy: 0.9590 - val_loss: 0.2500
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 229s 4s/step - accuracy: 0.9998 - loss: 9.9099e-04 - val_accuracy: 0.9535 - val_loss: 0.3037
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 229s 4s/step - accuracy: 1.0000 - loss: 7.7786e-04 - val_accuracy: 0.9634 - val_loss: 0.2132
